In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [13]:
class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1
    
    def make_move(self, row, col):
        if self.board[row, col] == 0:
            self.board[row, col] = self.current_player
            self.current_player = 3 - self.current_player  # Switch player
            return True
        return False

    def check_winner(self):
        for i in range(3):
            if np.all(self.board[i, :] == self.current_player):
                return self.current_player
            if np.all(self.board[:, i] == self.current_player):
                return self.current_player
        
        if self.board[0, 0] == self.board[1, 1] == self.board[2, 2] == self.current_player:
            return self.current_player
        
        if self.board[0, 2] == self.board[1, 1] == self.board[2, 0] == self.current_player:
            return self.current_player
        
        return 0

    def is_draw(self):
        return np.all(self.board != 0)

    def is_game_over(self):
        return self.check_winner() or self.is_draw()

    def get_valid_moves(self):
        return [(i, j) for i in range(3) for j in range(3) if self.board[i, j] == 0]

    def display(self):
        print(self.board)

In [14]:
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = stride),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [15]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=9):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.fc1 = nn.Linear(128 * 3 * 3, num_classes)
        self.fc2 = nn.Linear(128 * 3 * 3, 1) # For winning probability

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        policy = self.fc1(out)
        value = torch.tanh(self.fc2(out))
        return policy, value

In [16]:
def ResNet18():
    return ResNet(ResNetBlock, [2, 2])

In [17]:
class MCTSNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = {}
        self.visits - 0
        self.value = 0.0
        self.prior = 0.0

    def is_leaf(self):
        return len(self.children) == 0
    
    def expand(self, action_priors):
        for action, prior, in action_priors:
            self.children[action] = MCTSNode(self.state, parent=self)
            self.children[action].prior = prior

In [18]:
class MCTS:
    def __init__(self, model, c_puct=1.0, n_playouts=1600):
        self.model = model
        self.c_puct = c_puct
        self.n_playouts = n_playouts

    def _uct_select(self, node):
        return max(node.children.items(), key=lambda act_node: act_node[1].value/(1 + act_node[1].visits) + self.c_puct * act_node[1].prior * math.sqrt(node.visits) / (1 + act_node[1].visits))
    
    def _playout(self, state):
        node = self.root
        while not node.is_leaf():
            action, node = self._uct_select(node)
            state.make_move(*action)
        action_probs, _ = self.model(torch.FloatTensor(state.board).unsqueeze(0).unsqueeze(0))
        action_probs = torch.softmax(action_probs, dim=1).detach().numpy().flatten()
        valid_moves = state.get_valid_moves()
        action_priors = [(move, action_probs[i]) for i, move in enumerate(valid_moves)]
        node.expand(action_priors) 

        leaf_value = self._evaluate(state)
        self._backpropagate(node, leaf_value)

    def _evaluate(self, state):
        _, value = self.model(torch.FloatTensor(state.board).unsqueeze(0).unsqueeze(0))
        return value.item()
    
    def _backpropagate(self, node, value):
        while node:
            node.visits += 1
            node.value += value
            node = node.parent

    def get_move(self, state):
        self.root = MCTSNode(state)
        for _ in range(self.n_playouts):
            self._playout(state)
        return max(self.root.children.items(), key=lambda act_node: act_node[1].visits)[0]

In [19]:
def self_play(game, model, mcts, n_games=1000):
    data = []
    for _ in range(n_games):
        state = game()
        while not state.is_game_over():
            move = mcts.get_move(state)
            data.append((state.board.copy(), move))
            state.make_move(*move)
        winner = state.check_winner()
        for board, move in data:
            if winner == state.current_player:
                reward = 1
            elif winner == 0:
                reward = 0
            else: 
                reward = -1
            yield board, move, reward

In [20]:
def train(model, optimizer, game, mcts, n_games=1000):
    model.train()
    for board, move, reward in self_play(game, model, mcts, n_games):
        optimizer.zero_grad()
        board = torch.FloatTensor(board).unsqueeze(0).unsqueeze(0)
        policy, value = model(board)
        loss = F.mse_loss(value, torch.FloatTensor([reward])) + F.cross_entropy(policy, torch.LongTensor([move]))
        loss.backward()
        optimizer.step()